# Chart and map urbanization data

In [1]:
import sys, os, importlib, shutil, pathlib, datetime, multiprocessing
import requests

import rasterio, elevation, richdem
import rasterio.warp
import pandas as pd
import geopandas as gpd
import numpy as np

from rasterio import features
from datetime import datetime
from shapely.geometry import MultiPolygon, Polygon, box, Point
from osgeo import gdal

#Import raster helpers
sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")

import GOSTRocks.rasterMisc as rMisc
from GOSTRocks.misc import tPrint

#Import GOST urban functions
sys.path.append("../../../src")
import GOST_Urban.UrbanRaster as urban
import GOST_Urban.urban_helper as helper

#Import local functions
import novelUrbanization as nu
from novelUrbanization import *

%load_ext autoreload
%autoreload 2

In [2]:
base_folder = "/home/wb411133/data/Projects/MR_Novel_Urbanization/Data"
pop_layer = 'cpo15'
aapc_folder = os.path.join(base_folder, "AAPPC", 'Delineations')
agg_folder = f"/home/wb411133/data/Projects/MR_Novel_Urbanization/Aggregate_Data/{pop_layer}"
if not os.path.exists(agg_folder):
        os.makedirs(agg_folder)
     
# Generate a list of iso3 codes already processed
processed_iso3 = []
for folder in os.listdir(base_folder):
    if "URBAN_DATA_new" in folder:
        processed_iso3.append(folder[:3])
print(processed_iso3)

['BGD', 'AGO', 'COL', 'EGY', 'ETH', 'GHA', 'TZA', 'VNM', 'BDI', 'BEN', 'BFA', 'BWA', 'CAF', 'CIV', 'CMR', 'COD', 'COG', 'COM', 'CPV', 'ERI', 'GAB', 'GIN', 'GMB', 'GNB', 'KEN', 'LBR', 'LSO', 'MDG', 'MLI', 'MOZ', 'MRT', 'MUS', 'MWI', 'NAM', 'NER', 'NGA', 'RWA', 'SDN', 'SEN', 'SLE', 'SOM', 'SSD', 'STP', 'SWZ', 'SYC', 'TCD', 'TGO', 'UGA', 'ZAF', 'ZMB', 'ZWE', 'MAU', 'MAR', 'DZA', 'TUN', 'LBY', 'PAK']


In [3]:
iso3 = 'KEN'
country_folder = os.path.join(base_folder, f'{iso3}_URBAN_DATA_new_naming')
urb = urban_data(iso3, country_folder, aapc_folder)
comboRes = urb.generate_combo_layer(pop_type=pop_layer, debug=True)
if comboRes:
    urb.write_results(comboRes, agg_folder, dbhd = 'co')
tPrint(iso3)

/home/wb411133/data/Projects/MR_Novel_Urbanization/Data/KEN_URBAN_DATA_new_naming/FINAL_STANDARD/ken_cpo15_urban.tif
/home/wb411133/data/Projects/MR_Novel_Urbanization/Data/KEN_URBAN_DATA_new_naming/FINAL_STANDARD/ken_cpo15_urban_hd.tif
/home/wb411133/data/Projects/MR_Novel_Urbanization/Data/AAPPC/Delineations/ken_cpo15d10b3000_cc.tif
/home/wb411133/data/Projects/MR_Novel_Urbanization/Data/AAPPC/Delineations/ken_cpo15d10b3000_co.tif
/home/wb411133/data/Projects/MR_Novel_Urbanization/Data/AAPPC/Delineations/ken_cpo15d10b3000_ur.tif
09:40:48	KEN


In [ ]:
with multiprocessing.Pool(len(processed_iso3)) as mp:
    mp.map(multiP, processed_iso3)

In [ ]:
sum_files = []
bin_files = []

for root, dirs, files in os.walk(agg_folder):
    for f in files:
        if f.endswith("sum.tif"):
            sum_files.append(os.path.join(root, f))
        if f.endswith("binary.tif"):
            bin_files.append(os.path.join(root, f))

In [ ]:
vrt_options = gdal.BuildVRTOptions(resampleAlg='cubic', addAlpha=True)
pop_layer = 'cpo'
my_vrt = gdal.BuildVRT(f'{agg_folder}_{pop_layer}_sum.vrt', sum_files, options=vrt_options)
my_vrt = None
my_vrt = gdal.BuildVRT(f'{agg_folder}_{pop_layer}_binary.vrt', bin_files, options=vrt_options)
my_vrt = None

In [ ]:
f'{agg_folder}_{pop_layer}_sum.vrt'

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from matplotlib.colors import ListedColormap
plotD = urbanD[0,:,:]

In [ ]:
# Plot the resulting areas using matplotlib
fig, ax = plt.subplots(figsize=(round(plotD.shape[0]/100),round(plotD.shape[1]/100)))
ax.imshow(plotD)